In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import torch
# print("Pytorch version：")
# print(torch.__version__)
# print("CUDA Version: ")
# print(torch.version.cuda)
# print("cuDNN version is :")
# print(torch.backends.cudnn.version())

In [4]:
!pip install -q torch_geometric
!pip install -q git+https://github.com/GrantMcConachie/pytorch_geometric_temporal.git 

In [4]:
# !pip install torch_geometric #--target=/kaggle/working/
# # !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu123.html 

# !pip install git+https://github.com/GrantMcConachie/pytorch_geometric_temporal.git 

# Converting tabular dataset into a graph compatible format
> We have a tabular dataset that needs to be converted to a form compatitble with pytorch geometric temporal.
> 1. Identify node and edge level matrix
> 2. Decide time steps and map ids to location columns
> 3. Create feature matrix


In [7]:
df = pd.read_excel('dataset/mod_dataset.xlsx')

In [8]:
df['Area Name'].unique()

array(['Indiranagar', 'Whitefield', 'Koramangala', 'M.G. Road',
       'Jayanagar', 'Hebbal', 'Yeshwanthpur', 'Electronic City'],
      dtype=object)

In [7]:
df['Road/Intersection Name'].unique()

array(['100 Feet Road', 'CMH Road', 'Marathahalli Bridge',
       'Sony World Junction', 'Sarjapur Road', 'Trinity Circle',
       'Anil Kumble Circle', 'Jayanagar 4th Block', 'South End Circle',
       'Hebbal Flyover', 'Ballari Road', 'Yeshwanthpur Circle',
       'Tumkur Road', 'ITPL Main Road', 'Silk Board Junction',
       'Hosur Road'], dtype=object)

In [8]:
df.dtypes

Date                      datetime64[ns]
Area Name                         object
Road/Intersection Name            object
Traffic Volume                     int64
Average Speed                    float64
                               ...      
Embed_763                        float64
Embed_764                        float64
Embed_765                        float64
Embed_766                        float64
Embed_767                        float64
Length: 784, dtype: object

In [9]:
from datetime import datetime, timedelta
print(df['Date'].iloc[0] + timedelta(days = 1))

2022-01-02 00:00:00


In [10]:
df.columns[3:]

Index(['Traffic Volume', 'Average Speed', 'Travel Time Index',
       'Congestion Level', 'Road Capacity Utilization', 'Incident Reports',
       'Environmental Impact', 'Public Transport Usage',
       'Traffic Signal Compliance', 'Parking Usage',
       ...
       'Embed_758', 'Embed_759', 'Embed_760', 'Embed_761', 'Embed_762',
       'Embed_763', 'Embed_764', 'Embed_765', 'Embed_766', 'Embed_767'],
      dtype='object', length=781)

### Mapping Area Name and Road/Intersection names into numeric codes

In [10]:
areas = df['Area Name'].unique()
new_ids = list(range(len(df['Area Name'].unique())))
mapping = dict(zip(areas, new_ids))

df['Area Name'] = df['Area Name'].map(mapping)

df.head()

,Date,Area Name,Road/Intersection Name,Traffic Volume,Average Speed,Travel Time Index,Congestion Level,Road Capacity Utilization,Incident Reports,Environmental Impact,...,Embed_758,Embed_759,Embed_760,Embed_761,Embed_762,Embed_763,Embed_764,Embed_765,Embed_766,Embed_767
0,2022-01-01,0,100 Feet Road,50590,50.230299,1.500000,100.000000,100.000000,0,151.180,...,-0.121476,0.060294,0.198639,-0.037079,-0.118171,-0.460287,0.074908,-0.251739,0.086459,-0.358863
1,2022-01-01,0,CMH Road,30825,29.377125,1.500000,100.000000,100.000000,1,111.650,...,-0.121476,0.060294,0.198639,-0.037079,-0.118171,-0.460287,0.074908,-0.251739,0.086459,-0.358863
2,2022-01-01,1,Marathahalli Bridge,7399,54.474398,1.039069,28.347994,36.396525,0,64.798,...,-0.136313,0.041506,0.197209,-0.186619,-0.045450,-0.544922,-0.044758,-0.181581,0.202162,-0.250891
3,2022-01-01,2,Sony World Junction,60874,43.817610,1.500000,100.000000,100.000000,1,171.748,...,-0.066145,-0.060249,0.252572,-0.012830,-0.235883,-0.400892,-0.023987,-0.387391,0.102918,-0.194830
4,2022-01-01,2,Sarjapur Road,57292,41.116763,1.500000,100.000000,100.000000,3,164.584,...,-0.066145,-0.060249,0.252572,-0.012830,-0.235883,-0.400892,-0.023987,-0.387391,0.102918,-0.194830


In [11]:
intersections = df['Road/Intersection Name'].unique()
new_ids1 = list(range(len(df['Road/Intersection Name'].unique())))
mapping1 = dict(zip(intersections, new_ids1))

df['Road/Intersection Name'] = df['Road/Intersection Name'].map(mapping1)

df.head()

,Date,Area Name,Road/Intersection Name,Traffic Volume,Average Speed,Travel Time Index,Congestion Level,Road Capacity Utilization,Incident Reports,Environmental Impact,...,Embed_758,Embed_759,Embed_760,Embed_761,Embed_762,Embed_763,Embed_764,Embed_765,Embed_766,Embed_767
0,2022-01-01,0,0,50590,50.230299,1.500000,100.000000,100.000000,0,151.180,...,-0.121476,0.060294,0.198639,-0.037079,-0.118171,-0.460287,0.074908,-0.251739,0.086459,-0.358863
1,2022-01-01,0,1,30825,29.377125,1.500000,100.000000,100.000000,1,111.650,...,-0.121476,0.060294,0.198639,-0.037079,-0.118171,-0.460287,0.074908,-0.251739,0.086459,-0.358863
2,2022-01-01,1,2,7399,54.474398,1.039069,28.347994,36.396525,0,64.798,...,-0.136313,0.041506,0.197209,-0.186619,-0.045450,-0.544922,-0.044758,-0.181581,0.202162,-0.250891
3,2022-01-01,2,3,60874,43.817610,1.500000,100.000000,100.000000,1,171.748,...,-0.066145,-0.060249,0.252572,-0.012830,-0.235883,-0.400892,-0.023987,-0.387391,0.102918,-0.194830
4,2022-01-01,2,4,57292,41.116763,1.500000,100.000000,100.000000,3,164.584,...,-0.066145,-0.060249,0.252572,-0.012830,-0.235883,-0.400892,-0.023987,-0.387391,0.102918,-0.194830


In [13]:
df[df.columns[3:]]

,Traffic Volume,Average Speed,Travel Time Index,Congestion Level,Road Capacity Utilization,Incident Reports,Environmental Impact,Public Transport Usage,Traffic Signal Compliance,Parking Usage,...,Embed_758,Embed_759,Embed_760,Embed_761,Embed_762,Embed_763,Embed_764,Embed_765,Embed_766,Embed_767
0,50590,50.230299,1.500000,100.000000,100.000000,0,151.180,70.632330,84.044600,85.403629,...,-0.121476,0.060294,0.198639,-0.037079,-0.118171,-0.460287,0.074908,-0.251739,0.086459,-0.358863
1,30825,29.377125,1.500000,100.000000,100.000000,1,111.650,41.924899,91.407038,59.983689,...,-0.121476,0.060294,0.198639,-0.037079,-0.118171,-0.460287,0.074908,-0.251739,0.086459,-0.358863
2,7399,54.474398,1.039069,28.347994,36.396525,0,64.798,44.662384,61.375541,95.466020,...,-0.136313,0.041506,0.197209,-0.186619,-0.045450,-0.544922,-0.044758,-0.181581,0.202162,-0.250891
3,60874,43.817610,1.500000,100.000000,100.000000,1,171.748,32.773123,75.547092,63.567452,...,-0.066145,-0.060249,0.252572,-0.012830,-0.235883,-0.400892,-0.023987,-0.387391,0.102918,-0.194830
4,57292,41.116763,1.500000,100.000000,100.000000,3,164.584,35.092601,64.634762,93.155171,...,-0.066145,-0.060249,0.252572,-0.012830,-0.235883,-0.400892,-0.023987,-0.387391,0.102918,-0.194830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,33610,30.383013,1.500000,100.000000,100.000000,2,117.220,42.760049,85.782224,87.580813,...,-0.133175,0.057452,0.247932,-0.199149,0.048471,-0.360806,-0.204738,-0.129480,-0.007499,-0.084982
1615,25893,33.202016,1.500000,87.776516,100.000000,0,101.786,42.212279,76.763141,80.010309,...,-0.133175,0.057452,0.247932,-0.199149,0.048471,-0.360806,-0.204738,-0.129480,-0.007499,-0.084982
1616,31933,43.258556,1.500000,100.000000,100.000000,1,113.866,23.753016,78.303965,93.643966,...,-0.005167,0.039744,0.248549,-0.101699,-0.125346,-0.604413,-0.001497,-0.218152,0.191725,-0.245286
1617,44464,37.517852,1.500000,100.000000,100.000000,1,138.928,45.596332,89.203161,84.919639,...,-0.091483,-0.030526,0.248418,-0.079809,-0.123212,-0.314797,-0.124355,-0.086510,0.196742,-0.196569


In [12]:
def classify_traffic(row):
    """
    Classify traffic as 'high' or 'low' based on multiple criteria.

    Criteria for high traffic:
    - Traffic Volume above 75th percentile
    - Average Speed below 25th percentile
    - Congestion Level above 75th percentile
    """
    volume_threshold = row['Traffic Volume'] > df['Traffic Volume'].quantile(0.75)
    speed_threshold = row['Average Speed'] < df['Average Speed'].quantile(0.25)
    congestion_threshold = row['Congestion Level'] > df['Congestion Level'].quantile(0.75)
    
    # Combine criteria with logical OR
    return 1 if (volume_threshold or speed_threshold or congestion_threshold) else 0

def prepare_data(df):
    """
    Add traffic classification column
    """
    # Add traffic classification column
    df['Traffic_Classification'] = df.apply(classify_traffic, axis=1)

prepare_data(df)

In [13]:
one_hot = pd.get_dummies(df['Weather Conditions'])

df = pd.concat([df, one_hot], axis=1)

df = df.drop('Weather Conditions', axis=1)

map = {'No': 0, 'Yes': 1}
df['Roadwork and Construction Activity'] = df['Roadwork and Construction Activity'].map(map)
# X['Roadwork and Construction Activity'] = X['animal'].apply(lambda x: format(x, 'b'))
df['Roadwork and Construction Activity']

0       0
1       0
2       0
3       0
4       0
       ..
1614    0
1615    0
1616    0
1617    0
1618    0
Name: Roadwork and Construction Activity, Length: 1619, dtype: int64

In [14]:
date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max())

# Create a complete DataFrame for all locations
complete_df = pd.MultiIndex.from_product([date_range, df['Road/Intersection Name'].unique()], names=['Date', 'Road/Intersection Name']).to_frame(index=False)

# Merge with the original DataFrame
df_new = pd.merge(complete_df, df, on=['Date', 'Road/Intersection Name'], how='left')

# Impute missing values
# df.fillna(method='bfill', inplace=True) 
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")

df_new = pd.DataFrame(imp.fit_transform(df_new), columns = df.columns)


In [17]:
df_new.head()

,Date,Area Name,Road/Intersection Name,Traffic Volume,Average Speed,Travel Time Index,Congestion Level,Road Capacity Utilization,Incident Reports,Environmental Impact,...,Embed_764,Embed_765,Embed_766,Embed_767,Traffic_Classification,Clear,Fog,Overcast,Rain,Windy
0,2022-01-01,0,0.0,50590.0,50.230299,1.5,100.0,100.0,0.0,151.18,...,0.074908,-0.251739,0.086459,-0.358863,1.0,True,False,False,False,False
1,2022-01-01,1,0.0,30825.0,29.377125,1.5,100.0,100.0,1.0,111.65,...,0.074908,-0.251739,0.086459,-0.358863,1.0,True,False,False,False,False
2,2022-01-01,2,1.0,7399.0,54.474398,1.039069,28.347994,36.396525,0.0,64.798,...,-0.044758,-0.181581,0.202162,-0.250891,0.0,True,False,False,False,False
3,2022-01-01,3,2.0,60874.0,43.81761,1.5,100.0,100.0,1.0,171.748,...,-0.023987,-0.387391,0.102918,-0.19483,1.0,True,False,False,False,False
4,2022-01-01,4,2.0,57292.0,41.116763,1.5,100.0,100.0,3.0,164.584,...,-0.023987,-0.387391,0.102918,-0.19483,1.0,True,False,False,False,False


In [18]:
df_new.shape

(2768, 789)

In [15]:
df_new.isnull().sum()

Date                      0
Area Name                 0
Road/Intersection Name    0
Traffic Volume            0
Average Speed             0
                         ..
Clear                     0
Fog                       0
Overcast                  0
Rain                      0
Windy                     0
Length: 789, dtype: int64

In [16]:
df = df_new.copy()

In [17]:
X = df_new.drop(columns=["Traffic_Classification", "Date"])
X

,Area Name,Road/Intersection Name,Traffic Volume,Average Speed,Travel Time Index,Congestion Level,Road Capacity Utilization,Incident Reports,Environmental Impact,Public Transport Usage,...,Embed_763,Embed_764,Embed_765,Embed_766,Embed_767,Clear,Fog,Overcast,Rain,Windy
0,0,0.0,50590.0,50.230299,1.5,100.0,100.0,0.0,151.18,70.63233,...,-0.460287,0.074908,-0.251739,0.086459,-0.358863,True,False,False,False,False
1,1,0.0,30825.0,29.377125,1.5,100.0,100.0,1.0,111.65,41.924899,...,-0.460287,0.074908,-0.251739,0.086459,-0.358863,True,False,False,False,False
2,2,1.0,7399.0,54.474398,1.039069,28.347994,36.396525,0.0,64.798,44.662384,...,-0.544922,-0.044758,-0.181581,0.202162,-0.250891,True,False,False,False,False
3,3,2.0,60874.0,43.81761,1.5,100.0,100.0,1.0,171.748,32.773123,...,-0.400892,-0.023987,-0.387391,0.102918,-0.19483,True,False,False,False,False
4,4,2.0,57292.0,41.116763,1.5,100.0,100.0,3.0,164.584,35.092601,...,-0.400892,-0.023987,-0.387391,0.102918,-0.19483,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,11,0.0,5435.0,20.0,1.5,100.0,100.0,1.0,60.87,10.00876,...,-0.665739,-0.335044,-0.395696,0.090292,-0.380956,True,False,False,False,False
2764,12,0.0,5435.0,20.0,1.5,100.0,100.0,1.0,60.87,10.00876,...,-0.665739,-0.335044,-0.395696,0.090292,-0.380956,True,False,False,False,False
2765,13,1.0,31933.0,43.258556,1.5,100.0,100.0,1.0,113.866,23.753016,...,-0.604413,-0.001497,-0.218152,0.191725,-0.245286,False,False,False,True,False
2766,14,0.0,5435.0,20.0,1.5,100.0,100.0,1.0,60.87,10.00876,...,-0.665739,-0.335044,-0.395696,0.090292,-0.380956,True,False,False,False,False


In [22]:
# one_hot = pd.get_dummies(X['Weather Conditions'])

# X = pd.concat([X, one_hot], axis=1)

# X = X.drop('Weather Conditions', axis=1)

# map = {'No': 0, 'Yes': 1}
# X['Roadwork and Construction Activity'] = X['Roadwork and Construction Activity'].map(map)
# # X['Roadwork and Construction Activity'] = X['animal'].apply(lambda x: format(x, 'b'))
# X['Roadwork and Construction Activity']

In [18]:
# Node feature matrix without embeddings
cols_to_drop = X.columns[X.columns.str.contains('Embed')]
cols_to_drop
X1 = X.drop(cols_to_drop, axis = 1)
node_features1 = np.stack(X1.values).astype(float)

In [19]:
node_features1.shape

(2768, 19)

In [20]:
# Node feature matrix with embeddings
node_features = np.stack(X.values).astype(float)
node_features.shape

(2768, 787)

In [26]:
node_features.dtype

dtype('float64')

## Creating Edges

Our dataset doesn't contain edge information. To get edges we obtained the coordinates of each road/intersection names through Google maps and then found the distance between all possible pair of locations. Finally we decide whether an edge exists between pairs of locations if their distance lie below a certain threshold.

In [27]:
intersections

array(['100 Feet Road', 'CMH Road', 'Marathahalli Bridge',
       'Sony World Junction', 'Sarjapur Road', 'Trinity Circle',
       'Anil Kumble Circle', 'Jayanagar 4th Block', 'South End Circle',
       'Hebbal Flyover', 'Ballari Road', 'Yeshwanthpur Circle',
       'Tumkur Road', 'ITPL Main Road', 'Silk Board Junction',
       'Hosur Road'], dtype=object)

In [21]:
coordinates = {
    
    'Road/Intersection Name':[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
    'latitude':[12.950872161822497,12.937344765607286,12.956812620966838,12.937295870568489,12.927611589940787,12.972770756681662,12.97678823893705,12.925867879823919,12.938556141793427,13.04331205962523,13.03764128918579,13.018796576148233,13.158490253705295,12.984091491454183,12.918304412683543,12.853857541045986],
    'longitude':[77.63919363777266,77.62693968912164, 77.70527903148383, 77.62694022225013, 77.62106437905125, 77.61949981185973,77.60179669544482,77.58283905081616, 77.58007369783098, 77.5906428396253,77.58932284346027, 77.55761794218522,  77.32624794580622,77.75145865240036,77.62302345751505,77.66645888532481]    
}

coordf = pd.DataFrame(coordinates)
coordf

,Road/Intersection Name,latitude,longitude
0,0,12.950872,77.639194
1,1,12.937345,77.626940
2,2,12.956813,77.705279
3,3,12.937296,77.626940
4,4,12.927612,77.621064
5,5,12.972771,77.619500
6,6,12.976788,77.601797
7,7,12.925868,77.582839
8,8,12.938556,77.580074
9,9,13.043312,77.590643


## Distances using Gmaps

In [ ]:
# coordinates = {
    
#     'Road/Intersection Name':[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
#     'latitude':[12.950872161822497,12.937344765607286,12.956812620966838,12.937295870568489,12.927611589940787,12.972770756681662,12.97678823893705,12.925867879823919,12.938556141793427,13.04331205962523,13.03764128918579,13.018796576148233,13.158490253705295,12.984091491454183,12.918304412683543,12.853857541045986],
#     'longitude':[77.63919363777266,77.62693968912164, 77.70527903148383, 77.62694022225013, 77.62106437905125, 77.61949981185973,77.60179669544482,77.58283905081616, 77.58007369783098, 77.5906428396253,77.58932284346027, 77.55761794218522,  77.32624794580622,77.75145865240036,77.62302345751505,77.66645888532481]    
# }

# coordf = pd.DataFrame(coordinates)
# coordf

,Road/Intersection Name,latitude,longitude
0,0,12.950872,77.639194
1,1,12.937345,77.626940
2,2,12.956813,77.705279
3,3,12.937296,77.626940
4,4,12.927612,77.621064
5,5,12.972771,77.619500
6,6,12.976788,77.601797
7,7,12.925868,77.582839
8,8,12.938556,77.580074
9,9,13.043312,77.590643


In [ ]:
# from sklearn.utils.extmath import cartesian
# from geopy.distance import geodesic

# # # Get all possible start locations and their geo info
# # subset = ["start station longitude", "start station latitude", "start station id"]
# # all_starts = df.drop_duplicates(subset="start station id", keep="first")[subset]
# # # Get all possible end locations and their geo info
# # subset = ["end station longitude", "end station latitude", "end station id"]
# # all_ends = df.drop_duplicates(subset="end station id", keep="first")[subset]
# # # Combine all combinations in one dataframe
# distance_matrix = coordf.merge(coordf, how="cross")
# distance_matrix["distance"] = distance_matrix.apply(lambda x: geodesic((x["latitude_x"], x["longitude_x"]), 
#                                                           (x["latitude_y"], x["longitude_y"])).meters, axis=1)
# distance_matrix.head()

,Road/Intersection Name_x,latitude_x,longitude_x,Road/Intersection Name_y,latitude_y,longitude_y,distance
0,0,12.950872,77.639194,0,12.950872,77.639194,0.000000
1,0,12.950872,77.639194,1,12.937345,77.626940,2001.905337
2,0,12.950872,77.639194,2,12.956813,77.705279,7200.633754
3,0,12.950872,77.639194,3,12.937296,77.626940,2005.914015
4,0,12.950872,77.639194,4,12.927612,77.621064,3239.125421


In [31]:
distance_matrix.shape

(256, 7)

In [ ]:
distance_matrix['distance'].mean()  

12468.794049296794

In [75]:
# Assigning edges to all nodes
distance_matrix["edge"] = True
distance_matrix

,Road/Intersection Name_x,latitude_x,longitude_x,Road/Intersection Name_y,latitude_y,longitude_y,distance,edge
0,0,12.950872,77.639194,0,12.950872,77.639194,0.000000,True
1,0,12.950872,77.639194,1,12.937345,77.626940,2001.905337,True
2,0,12.950872,77.639194,2,12.956813,77.705279,7200.633754,True
3,0,12.950872,77.639194,3,12.937296,77.626940,2005.914015,True
4,0,12.950872,77.639194,4,12.927612,77.621064,3239.125421,True
...,...,...,...,...,...,...,...,...
251,15,12.853858,77.666459,11,13.018797,77.557618,21735.915330,True
252,15,12.853858,77.666459,12,13.158490,77.326248,49979.100264,True
253,15,12.853858,77.666459,13,12.984091,77.751459,17107.557861,True
254,15,12.853858,77.666459,14,12.918304,77.623023,8547.332033,True


In [74]:
# Assigning edge basesd on distance threshold
# distance_matrix["edge"] = distance_matrix["distance"] < 1000
# distance_matrix.head()

In [76]:
distance_matrix[distance_matrix['edge'] == True]

,Road/Intersection Name_x,latitude_x,longitude_x,Road/Intersection Name_y,latitude_y,longitude_y,distance,edge
0,0,12.950872,77.639194,0,12.950872,77.639194,0.000000,True
1,0,12.950872,77.639194,1,12.937345,77.626940,2001.905337,True
2,0,12.950872,77.639194,2,12.956813,77.705279,7200.633754,True
3,0,12.950872,77.639194,3,12.937296,77.626940,2005.914015,True
4,0,12.950872,77.639194,4,12.927612,77.621064,3239.125421,True
...,...,...,...,...,...,...,...,...
251,15,12.853858,77.666459,11,13.018797,77.557618,21735.915330,True
252,15,12.853858,77.666459,12,13.158490,77.326248,49979.100264,True
253,15,12.853858,77.666459,13,12.984091,77.751459,17107.557861,True
254,15,12.853858,77.666459,14,12.918304,77.623023,8547.332033,True


In [77]:
# Use mask to extract static edges
edge_index = distance_matrix[distance_matrix["edge"] == True][["Road/Intersection Name_x", "Road/Intersection Name_y"]].values
edge_index = edge_index.transpose()
edge_index # [2 x num_edges]


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 

In [78]:
edge_index.shape

(2, 256)

In [79]:
# Add edge features to indicate edge type
distance_feature = distance_matrix[distance_matrix["edge"] == True]["distance"].values
static_edge_features = np.stack([distance_feature]).transpose()
static_edge_features # [num_edges x num_features]

array([[0.00000000e+00],
       [2.00190534e+03],
       [7.20063375e+03],
       [2.00591402e+03],
       [3.23912542e+03],
       [3.23034814e+03],
       [4.96832552e+03],
       [6.71168115e+03],
       [6.55812938e+03],
       [1.15033432e+04],
       [1.10190070e+04],
       [1.16101151e+04],
       [4.09836375e+04],
       [1.27229607e+04],
       [4.00752621e+03],
       [1.11331263e+04],
       [2.00190534e+03],
       [0.00000000e+00],
       [8.76902027e+03],
       [5.40956525e+00],
       [1.25137140e+03],
       [4.00145553e+03],
       [5.14624607e+03],
       [4.95112396e+03],
       [5.08727124e+03],
       [1.23668851e+04],
       [1.18225528e+04],
       [1.17373092e+04],
       [4.07706546e+04],
       [1.44665077e+04],
       [2.14887364e+03],
       [1.01834312e+04],
       [7.20063375e+03],
       [8.76902027e+03],
       [0.00000000e+00],
       [8.77029512e+03],
       [9.69233413e+03],
       [9.47301196e+03],
       [1.14431627e+04],
       [1.37199543e+04],


In [80]:
static_edge_features.shape

(256, 1)

In [81]:
#def extract_dynamic_edges(s):
#     # Extract dynamic edges and their features
#    trip_indices = s["Road/Intersection Name"].values
#     trip_durations = s["tripduration"]
#
#     # Build edge features
#     distance_feature  = pd.DataFrame(trip_indices, 
#                                    columns=["start station id", "end station id"]).merge(
#                                         distance_matrix, on=["start station id", "end station id"], 
#                                         how="left")["distance"].values
#     edge_type_feature = np.ones_like(distance_feature) # 1 = dynamic
#     trip_duration_feature = trip_durations
#     edge_features = np.stack([distance_feature, edge_type_feature, trip_duration_feature]).transpose()
#     return edge_features, trip_indices.transpose()



start_date = df['Date'].iloc[0]
end_date = df['Date'].iloc[-1]
interval = timedelta(days=1)

xs = [] # with embeddings
xs1 = [] # without embeddings
edge_indices = []
ys = []
y_indices = []
edge_features = []


while start_date < end_date:
    
    current_snapshot = df[df['Date'] == start_date]
    
    subsequent_snapshot = df[df['Date'] == (start_date + interval)]
    
    # Average duplicate df
    # current_snapshot = current_snapshot.groupby(["start station id", "end station id"]).mean().reset_index()
    # subsequent_snapshot = subsequent_snapshot.groupby(["start station id", "end station id"]).mean().reset_index()

    # Extract dynamic trip edges
    # edge_feats, additional_edge_index = extract_dynamic_edges(current_snapshot)
    # exteneded_edge_index = np.concatenate([edge_index, additional_edge_index], axis=1)
    # extended_edge_feats = np.concatenate([edge_feats, static_edge_features], axis=0)

    # Labels
    y = subsequent_snapshot["Traffic_Classification"].values
    y_index = subsequent_snapshot["Road/Intersection Name"].values

    # Append everything
    xs.append(node_features) # static
    xs1.append(node_features1)  
    # edge_indices.append(edge_index) # static + dynamic
    # edge_features.append(static_edge_features) # static + dynamic
    ys.append(y) # dynamic
    y_indices.append(y_index.transpose()) # dynamic

    # Increment
    start_date += interval

In [40]:
# i = 4
# print(f"""Example of graph snapshot {i}: \n
#       Node feature shape: {xs[i].shape} \n
#       Edge index shape: {edge_indices[i].shape} \n
#       Edge feature shape: {edge_features[i].shape} \n 
#       Labels shape: {ys[i].shape} \n
#       Labels mask shape: {y_indices[i].shape}
#       """)

In [54]:
edge_index = np.array(edge_index)
static_edge_features = np.array(static_edge_features)

In [82]:
edge_index

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 

In [83]:
i = 1
print(f"""Example of graph snapshot {i}: \n
      Node feature shape: {xs1[i].shape} \n
      Edge index shape: {edge_index.shape} \n
      Edge feature shape: {static_edge_features.shape} \n 
      Labels shape: {ys[i].shape} \n
      Labels mask shape: {y_indices[i].shape}
      """)

Example of graph snapshot 1: 

      Node feature shape: (2768, 19) 

      Edge index shape: (2, 256) 

      Edge feature shape: (256, 1) 
 
      Labels shape: (16,) 

      Labels mask shape: (16,)
      


In [84]:
y_indices = np.array(y_indices)
y_indices.shape

(172, 16)

## TGCN without embeddings for Classification

In [85]:
# from torch_geometric_temporal.signal import StaticGraphTemporalSignal
# dataset = StaticGraphTemporalSignal(
#             edge_index, static_edge_features, xs1, ys, y_indices=y_indices
#         )
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
dataset = StaticGraphTemporalSignal(
            edge_index, static_edge_features, xs1, ys, y_indices=y_indices
        )

In [87]:
import torch
from torch_geometric_temporal.nn.recurrent import TGCN
from torch.nn import Linear, functional as F
from sklearn.metrics import accuracy_score
from torch_geometric_temporal.signal import temporal_signal_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)


class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 64)
        self.linear1 = torch.nn.Linear(64, 32)
        self.linear2 = torch.nn.Linear(32, 1)  # Single output node for binary classification

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

model = RecurrentGCN(node_features=19)

# Change to BCEWithLogitsLoss for binary classification
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for epoch in tqdm(range(150)):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    hidden_state = None
    
    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, 
                                   snapshot.edge_attr, hidden_state)
        
        # Detach hidden state to break computational graph
        hidden_state = hidden_state.detach()
        
        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()
        
        loss = criterion(logits, targets)
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
        loss.backward()  # Now safe without retain_graph=True
        optimizer.step()
        optimizer.zero_grad()
    
    avg_loss = total_loss / (time + 1)
    accuracy = total_correct / total_samples
    # print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

print(f"Train Accuracy: {accuracy:.4f}")

# Evaluation loop
model.eval()
total_loss = 0
total_correct = 0
total_samples = 0
hidden_state = None

with torch.no_grad():
    for time, snapshot in enumerate(test_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index,
                                   snapshot.edge_attr, hidden_state)
        # No need to detach in eval but still reset hidden state
        hidden_state = hidden_state.detach()
        
        # logits = y_hat[snapshot.y_indices].flatten()
        logits = y_hat
        targets = snapshot.y.float()
        
        loss = criterion(logits, targets)
        total_loss += loss.item()
        
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
accuracy = total_correct / total_samples
print(f"Evaluation Accuracy: {accuracy:.4f}")
        

  0%|          | 0/150 [00:00<?, ?it/s]


AttributeError: 'GlobalStorage' object has no attribute 'y_indices'

In [88]:
print(train_dataset[1].y)    

None


## Rest of the notebook is testing

In [90]:
import torch
from torch_geometric_temporal.nn.recurrent import TGCN
from torch.nn import Linear, functional as F
from sklearn.metrics import accuracy_score
from torch_geometric_temporal.signal import temporal_signal_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 64)
        self.linear1 = torch.nn.Linear(64, 32)
        self.linear2 = torch.nn.Linear(32, 1)  # Single output node for binary classification

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

model = RecurrentGCN(node_features=19)

# Use BCEWithLogitsLoss for binary classification
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for epoch in tqdm(range(150)):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    hidden_state = None

    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, 
                                   snapshot.edge_attr, hidden_state)

        # Detach hidden state to break computational graph
        hidden_state = hidden_state.detach()

        # Remove indexing by y_indices and use the output directly
        logits = y_hat.flatten()
        targets = snapshot.y.float()

        loss = criterion(logits, targets)
        total_loss += loss.item()

        # Calculate accuracy
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)

        loss.backward()  # Now safe without retain_graph=True
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / (time + 1)
    accuracy = total_correct / total_samples
    # print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

print(f"Train Accuracy: {accuracy:.4f}")

# Evaluation loop
model.eval()
total_loss = 0
total_correct = 0
total_samples = 0
hidden_state = None

with torch.no_grad():
    for time, snapshot in enumerate(test_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index,
                                   snapshot.edge_attr, hidden_state)
        # Detach hidden state
        hidden_state = hidden_state.detach()

        logits = y_hat.flatten()
        targets = snapshot.y.float()

        loss = criterion(logits, targets)
        total_loss += loss.item()

        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)

accuracy = total_correct / total_samples
print(f"Evaluation Accuracy: {accuracy:.4f}")


  0%|          | 0/150 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'float'

In [91]:
for time, snapshot in enumerate(train_dataset):
    y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
    hidden_state = hidden_state.detach()

    # Check if snapshot has target labels
    if snapshot.y is None:
        raise ValueError(f"Snapshot at time {time} is missing labels. Please ensure your dataset provides a 'y' attribute.")



ValueError: Snapshot at time 0 is missing labels. Please ensure your dataset provides a 'y' attribute.

In [92]:
for time, snapshot in enumerate(train_dataset):
    print(snapshot.x.shape)
    print(snapshot.edge_index.shape)
    print(snapshot.edge_attr.shape)
    y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, 
                               snapshot.edge_attr, hidden_state)
    break

torch.Size([2768, 19])
torch.Size([2, 256])
torch.Size([256, 1])


## Testing changes

In [33]:
import torch
from torch_geometric_temporal.nn.recurrent import TGCN
from torch.nn import Linear, functional as F
from sklearn.metrics import accuracy_score
from torch_geometric_temporal.signal import temporal_signal_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)


class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 64,  add_self_loops=False)
        self.linear1 = torch.nn.Linear(64, 32)
        self.linear2 = torch.nn.Linear(32, 1)  

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

model = RecurrentGCN(node_features=19)

from torch_geometric.utils import add_self_loops

# Add self-loops to static graph structure once
edge_index, edge_attr = add_self_loops(
    dataset.edge_index, 
    edge_attr=dataset.edge_weight,
    fill_value=1.0,
    num_nodes=snapshot.x.size(0)# Set appropriate value for your domain
)

# Update all snapshots to use the same preprocessed edges
for snapshot in dataset:
    snapshot.edge_index = edge_index
    snapshot.edge_weight = edge_attr

# Change to BCEWithLogitsLoss for binary classification
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for epoch in tqdm(range(150)):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    hidden_state = None
    
    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, 
                                   snapshot.edge_weight, hidden_state)
        
        # Detach hidden state to break computational graph
        hidden_state = hidden_state.detach()
        
        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()
        
        loss = criterion(logits, targets)
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
        loss.backward()  # Now safe without retain_graph=True
        optimizer.step()
        optimizer.zero_grad()
    
    avg_loss = total_loss / (time + 1)
    accuracy = total_correct / total_samples
    # print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

print(f"Train Accuracy: {accuracy:.4f}")

# Evaluation loop
model.eval()
total_loss = 0
total_correct = 0
total_samples = 0
hidden_state = None

with torch.no_grad():
    for time, snapshot in enumerate(test_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index,
                                   snapshot.edge_attr, hidden_state)
        # No need to detach in eval but still reset hidden state
        hidden_state = hidden_state.detach()
        
        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()
        
        loss = criterion(logits, targets)
        total_loss += loss.item()
        
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
accuracy = total_correct / total_samples
print(f"Evaluation Accuracy: {accuracy:.4f}")
        

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

## Another fix

In [ ]:
import torch
from torch_geometric_temporal.nn.recurrent import TGCN
from torch.nn import Linear, functional as F
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch_geometric_temporal.signal import temporal_signal_split
from tqdm import tqdm

# Assuming 'dataset' is already loaded as a StaticTemporalSignal
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 64)
        self.linear1 = torch.nn.Linear(64, 32)
        self.linear2 = torch.nn.Linear(32, 1)  # Single output node for binary classification

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        # Pass a 1D edge weight tensor rather than a column vector.
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

model = RecurrentGCN(node_features=19)

# Using BCEWithLogitsLoss for binary classification
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for epoch in tqdm(range(150)):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    hidden_state = None

    for time, snapshot in enumerate(train_dataset):
        # Squeeze the edge attributes to convert shape from (E, 1) to (E,)
        edge_weight = snapshot.edge_attr
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, edge_weight, hidden_state)        # Detach hidden state to break computational graph
        hidden_state = hidden_state.detach()

        # Index predictions for the labeled nodes
        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()

        loss = criterion(logits, targets)
        total_loss += loss.item()

        # Calculate accuracy
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)

        loss.backward()  # Now safe without retain_graph=True
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / (time + 1)
    accuracy = total_correct / total_samples
    print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

# Evaluation loop
model.eval()
total_loss = 0
total_correct = 0
total_samples = 0
hidden_state = None

with torch.no_grad():
    for time, snapshot in enumerate(test_dataset):
        edge_weight = snapshot.edge_attr.squeeze(-1)
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, edge_weight, hidden_state)
        hidden_state = hidden_state.detach()

        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()

        loss = criterion(logits, targets)
        total_loss += loss.item()

        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)

    accuracy = total_correct / total_samples
    print(f"Evaluation Accuracy: {accuracy:.4f}")


# Trying out regression

In [91]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
dataset = DynamicGraphTemporalSignal(
            edge_indices, edge_features, xs1, ys, y_indices=y_indices
        )

try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import TGCN


from torch_geometric_temporal.signal import temporal_signal_split
train_dataset1, test_dataset1 = temporal_signal_split(dataset, train_ratio=0.2)

class RecurrentGCN1(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN1, self).__init__()
        self.recurrent = TGCN(node_features, 64)  # Increased hidden dimensions
        self.linear1 = torch.nn.Linear(64, 32)   # Added a hidden layer
        self.linear2 = torch.nn.Linear(32, 1)    # Output layer

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)                           # Activation
        h = F.dropout(h, p=0.2, training=self.training)  # Add dropout
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

# class RecurrentGCN1(torch.nn.Module):
#     def __init__(self, node_features):
#         super(RecurrentGCN1, self).__init__()
#         self.recurrent = TGCN(node_features, 32)
#         self.linear = torch.nn.Linear(32, 1)

#     def forward(self, x, edge_index, edge_weight, prev_hidden_state):
#         h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
#         y = F.relu(h)
#         y = self.linear(y)
#         return y, h
        
model1 = RecurrentGCN1(node_features = 19)

optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.01)

model1.train()

for epoch in tqdm(range(150)):
    cost = 0
    hidden_state = None
    for time, snapshot in enumerate(train_dataset1):
        y_hat, hidden_state = model1(snapshot.x, snapshot.edge_index, snapshot.edge_attr,hidden_state)
        print(y_hat)
        print(snapshot.y)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer1.step()
    optimizer1.zero_grad()
    
model1.eval()
cost = 0
hidden_state = None
for time, snapshot in enumerate(test_dataset1):
    y_hat, hidden_state = model1(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

  0%|          | 0/150 [00:00<?, ?it/s]

tensor([[ 0.1399],
        [ 0.0070],
        [ 0.1409],
        ...,
        [ 0.0436],
        [-0.0517],
        [-0.0479]], grad_fn=<AddmmBackward0>)
None


TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

In [ ]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal

dataset1 = DynamicGraphTemporalSignal(
            edge_indices, edge_features, xs, ys, y_indices=y_indices
        )

In [75]:
import torch
from torch_geometric_temporal.nn.recurrent import TGCN
from torch.nn import Linear, functional as F
from sklearn.metrics import accuracy_score
from torch_geometric_temporal.signal import temporal_signal_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

train_dataset, test_dataset = temporal_signal_split(dataset1, train_ratio=0.8)


class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 64)
        self.linear1 = torch.nn.Linear(64, 32)
        self.linear2 = torch.nn.Linear(32, 1)  # Single output node for binary classification

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)
        h = F.dropout(h, p=0.2, training=self.training)
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

model = RecurrentGCN(node_features=787)

# Change to BCEWithLogitsLoss for binary classification
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for epoch in tqdm(range(50)):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    hidden_state = None
    
    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, 
                                   snapshot.edge_attr, hidden_state)
        
        # Detach hidden state to break computational graph
        hidden_state = hidden_state.detach()
        
        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()
        
        loss = criterion(logits, targets)
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
        loss.backward()  # Now safe without retain_graph=True
        optimizer.step()
        optimizer.zero_grad()
    
    avg_loss = total_loss / (time + 1)
    accuracy = total_correct / total_samples
    # print(f"Epoch {epoch+1}: Train Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

print(f"Train Accuracy: {accuracy:.4f}")

# Evaluation loop
model.eval()
total_loss = 0
total_correct = 0
total_samples = 0
hidden_state = None

with torch.no_grad():
    for time, snapshot in enumerate(test_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index,
                                   snapshot.edge_attr, hidden_state)
        # No need to detach in eval but still reset hidden state
        hidden_state = hidden_state.detach()
        
        logits = y_hat[snapshot.y_indices].flatten()
        targets = snapshot.y.float()
        
        loss = criterion(logits, targets)
        total_loss += loss.item()
        
        preds = (torch.sigmoid(logits) > 0.5).float()
        total_correct += (preds == targets).sum().item()
        total_samples += targets.size(0)
        
accuracy = total_correct / total_samples
print(f"Evaluation Accuracy: {accuracy:.4f}")


  0%|          | 0/50 [00:00<?, ?it/s]


AttributeError: 'GlobalStorage' object has no attribute 'y_indices'

-------------------------------------------

ValueError: Found input variables with inconsistent numbers of samples: [5, 1619]

In [55]:
import torch
from torch_geometric_temporal.nn.recurrent import TGCN
from torch.nn import Linear, functional as F
from sklearn.metrics import accuracy_score
from torch_geometric_temporal.signal import temporal_signal_split


# Define the TGCN Model
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, hidden_channels, output_channels):
        super(TemporalGNN, self).__init__()
        self.tgcn = TGCN(node_features, hidden_channels)
        self.linear = Linear(hidden_channels, output_channels)

    def forward(self, x, edge_index, edge_weight, h):
        h = self.tgcn(x, edge_index, edge_weight, h)
        out = self.linear(h)
        return out, h


# Training and evaluation functions
def train_model(model, dataset, epochs, device):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    model.train()
    h = None  # Hidden state for the temporal model

    for epoch in range(epochs):
        total_loss = 0
        for snapshot in dataset:
            x, edge_index, edge_weight, y, y_indices = (
                snapshot.x.to(device),
                snapshot.edge_index.to(device),
                snapshot.edge_attr.to(device) if snapshot.edge_attr is not None else None,
                snapshot.y.to(device),
                snapshot.y_indices.to(device),
            )

            # Forward pass
            optimizer.zero_grad()
            out, h = model(x, edge_index, edge_weight, h)
            h = h.detach()  # Detach the hidden state for the next snapshot

            # Get predictions for nodes with labels
            out = out[y_indices]
            y = ys[y_indices]

            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

    print("Training complete!")


def evaluate_model(model, dataset, device):
    model.eval()
    all_preds = []
    all_labels = []

    h = None
    with torch.no_grad():
        for snapshot in dataset:
            x, edge_index, edge_weight, y, y_indices = (
                snapshot.x.to(device),
                snapshot.edge_index.to(device),
                snapshot.edge_attr.to(device) if snapshot.edge_attr is not None else None,
                snapshot.y.to(device),
                snapshot.y_index.to(device),
            )

            out, h = model(x, edge_index, edge_weight, h)
            out = out[y_indices]
            y = ys[y_indices]

            preds = out.argmax(dim=-1).cpu()
            all_preds.append(preds)
            all_labels.append(y.cpu())

    # Flatten lists and calculate accuracy
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    accuracy = accuracy_score(all_labels.numpy(), all_preds.numpy())
    print(f"Test Accuracy: {accuracy:.4f}")



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model parameters
node_features = 19  # Number of input node features
hidden_channels = 32  # Number of hidden units in TGCN
# output_channels = dataset.targets[0].max().item() + 1  # Number of classes
output_channels = 1  # Number of classes

# Model initialization
model = TemporalGNN(node_features, hidden_channels, output_channels)

# Split dataset into train and test
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)


# Train and evaluate
train_model(model, train_dataset, epochs=50, device=device)
evaluate_model(model, test_dataset, device=device)


IndexError: Target 1 is out of bounds.

In [40]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import TGCN
from torch_geometric_temporal.signal import temporal_signal_split
from tqdm import tqdm

# Assuming 'dataset' is already defined and loaded
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, hidden_units, num_classes):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, hidden_units)
        self.linear = torch.nn.Linear(hidden_units, num_classes)

    def forward(self, x, edge_index, edge_weight, h):
        h = self.recurrent(x, edge_index, edge_weight, h)
        h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        y = self.linear(h)
        return y, h

node_features = 19 # e.g., 787
hidden_units = 32
num_classes = 1  # e.g., 10

model = RecurrentGCN(node_features, hidden_units, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
model.train()
for epoch in tqdm(range(50)):
    total_loss = 0
    h = None
    for time, snapshot in enumerate(train_dataset):
        optimizer.zero_grad()
        out, h = model(x, edge_index, edge_weight, h)
        h = h.detach()
        out_labeled = out[y_indices]  # Select predictions for labeled nodes
        y_labeled = y[y_indices]  # Select corresponding labels
        loss = criterion(out_labeled, y_labeled)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / (time + 1)
    print(f'Epoch {epoch+1}, Loss: {avg_loss:.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
h = None
with torch.no_grad():
    for time, snapshot in enumerate(test_dataset):
        y_hat, h = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, h)
        predicted = y_hat.argmax(dim=1)
        correct += (predicted == snapshot.y).sum().item()
        total += snapshot.y.size(0)
accuracy = correct / total
print(f'Accuracy: {accuracy:.4f}')

  0%|          | 0/50 [00:00<?, ?it/s]


NameError: name 'x' is not defined

## Trying to make the TGCN do classification:
We'll modify the network and change the criterion for calculating the loss.

In [76]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import TGCN


from torch_geometric_temporal.signal import temporal_signal_split


train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 32)
        self.linear1 = torch.nn.Linear(32, 16)   # Added a hidden layer
        self.linear2 = torch.nn.Linear(16, 1)    # Output layer

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)                           # Activation
        h = F.dropout(h, p=0.2, training=self.training)  # Add dropout
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h
        
model = RecurrentGCN(node_features = 19)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

model.train()

for epoch in tqdm(range(50)):
    cost = 0
    hidden_state = None
    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr,hidden_state)
        # cost = cost + torch.mean((y_hat-snapshot.y)**2)
        cost = cost + criterion(y_hat[snapshot.y_indices].flatten(), snapshot.y)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    
model.eval()
cost = 0
hidden_state = None
for time, snapshot in enumerate(test_dataset):
    y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
    cost = cost + criterion(y_hat, snapshot.y)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

  0%|          | 0/50 [00:00<?, ?it/s]


IndexError: The shape of the mask [2, 20] at index 0 does not match the shape of the indexed tensor [172, 20, 1] at index 0

In [75]:
criterion(y_hat[snapshot.y_indices], snapshot.y)

IndexError: Target 1 is out of bounds.

In [77]:
y_hat[snapshot.y_indices].flatten()

AttributeError: 'GlobalStorage' object has no attribute 'y_indices'

In [95]:
snapshot.y.argmax(dim=-1)

tensor(1)

In [ ]:
# try:
#     from tqdm import tqdm
# except ImportError:
#     def tqdm(iterable):
#         return iterable

# import torch
# import torch.nn.functional as F
# from torch_geometric_temporal.nn.recurrent import TGCN


# from torch_geometric_temporal.signal import temporal_signal_split


# train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

# class RecurrentGCN(torch.nn.Module):
#     def __init__(self, node_features):
#         super(RecurrentGCN, self).__init__()
#         self.recurrent = TGCN(node_features, 32)
#         self.linear1 = torch.nn.Linear(64, 32)   # Added a hidden layer
#         self.linear2 = torch.nn.Linear(32, 1)    # Output layer

#     def forward(self, x, edge_index, edge_weight, prev_hidden_state):
#         h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
#         h = F.relu(h)                           # Activation
#         h = F.dropout(h, p=0.2, training=self.training)  # Add dropout
#         y = F.relu(self.linear1(h))
#         y = self.linear2(y)
#         return y, h
        
# model = RecurrentGCN(node_features = 19)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.CrossEntropyLoss()

# model.train()

# for epoch in tqdm(range(50)):
#     cost = 0
#     hidden_state = None
#     for time, snapshot in enumerate(train_dataset):
#         y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr,hidden_state)
#         # cost = cost + torch.mean((y_hat-snapshot.y)**2)
#         cost = cost + criterion(y_hat, snapshot.y)
#     cost = cost / (time+1)
#     cost.backward()
#     optimizer.step()
#     optimizer.zero_grad()
    
# model.eval()
# cost = 0
# hidden_state = None
# for time, snapshot in enumerate(test_dataset):
#     y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
#     cost = cost + criterion(y_hat, snapshot.y)
# cost = cost / (time+1)
# cost = cost.item()
# print("MSE: {:.4f}".format(cost))

In [92]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
dataset1 = DynamicGraphTemporalSignal(
            edge_indices, edge_features, xs1, ys, y_indices=y_indices
        )

In [95]:
train_dataset1, test_dataset1 = temporal_signal_split(dataset1, train_ratio=0.2)

In [98]:
for time, snapshot in enumerate(train_dataset):
    print(snapshot.x.shape)
    print(snapshot.edge_index.shape)
    print(snapshot.edge_attr.shape)
    break

torch.Size([2768, 19])
torch.Size([172, 2, 20])
torch.Size([172, 20, 1])


In [97]:
for time, snapshot in enumerate(train_dataset1):
    print(snapshot.x.shape)
    print(snapshot.edge_index.shape)
    print(snapshot.edge_attr.shape)
    break
       

torch.Size([2768, 19])
torch.Size([2, 20])
torch.Size([20, 1])


In [ ]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import TGCN


from torch_geometric_temporal.signal import temporal_signal_split
train_dataset1, test_dataset1 = temporal_signal_split(dataset1, train_ratio=0.2)

class RecurrentGCN1(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN1, self).__init__()
        self.recurrent = TGCN(node_features, 64)  # Increased hidden dimensions
        self.linear1 = torch.nn.Linear(64, 32)   # Added a hidden layer
        self.linear2 = torch.nn.Linear(32, 1)    # Output layer

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        h = F.relu(h)                           # Activation
        h = F.dropout(h, p=0.2, training=self.training)  # Add dropout
        y = F.relu(self.linear1(h))
        y = self.linear2(y)
        return y, h

# class RecurrentGCN1(torch.nn.Module):
#     def __init__(self, node_features):
#         super(RecurrentGCN1, self).__init__()
#         self.recurrent = TGCN(node_features, 32)
#         self.linear = torch.nn.Linear(32, 1)

#     def forward(self, x, edge_index, edge_weight, prev_hidden_state):
#         h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
#         y = F.relu(h)
#         y = self.linear(y)
#         return y, h
        
model1 = RecurrentGCN1(node_features = 18)

optimizer1 = torch.optim.Adam(model1.parameters(), lr=0.01)

model1.train()

for epoch in tqdm(range(200)):
    cost = 0
    hidden_state = None
    for time, snapshot in enumerate(train_dataset1):
        y_hat, hidden_state = model1(snapshot.x, snapshot.edge_index, snapshot.edge_attr,hidden_state)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer1.step()
    optimizer1.zero_grad()
    
model1.eval()
cost = 0
hidden_state = None
for time, snapshot in enumerate(test_dataset1):
    y_hat, hidden_state = model1(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))